## Predictions of BTC-USD Price Using SBi-LSTM and SBi-GRU

In [ ]:
# load all functions
from C01_visualization import *
from C02_model_predictions import *
from C03_model_evaluate import *

In [ ]:
# lib manipulation data
import numpy as np
import pandas as pd

# lib data visualizations
import seaborn as sns
import matplotlib.pyplot as plt

# lib data preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

- config models

In [ ]:
# set random number
import random as rm
rm.seed(1234)

# set random number
import numpy as np
np.random.seed(1234)

# set random number
import tensorflow as tf
tf.random.set_seed(1234)

### 1. Data Collections

In [ ]:
# load dataset
dataset = pd.read_csv("../dataset/Cryptocurrency-BTC-USD-2024-04.csv", parse_dates=['Date'])
dataset = dataset[["Date", "Open", "High", "Low", "Close"]]
dataset.info()

In [ ]:
# show dataset
print(dataset.tail())

### 2. Exploration Data Analysis

In [ ]:
timeseries_matplotlib(dataset, ["Open", "High", "Low", "Close"])

### 3. Data Preprocessing

In [ ]:
# 1. process feature selection
data = dataset.filter(['Close'])
data = data.values

In [ ]:
# 1. results feature selection
np.round(data[:5],6)

In [ ]:
# 2. process normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(np.array(data).reshape(-1,1))

In [ ]:
# 2. results normalize features
np.round(scaled[:5],6)

In [ ]:
# 2. results normalize features
lineplot_matplotlib1(
  x1=dataset["Date"], y1=scaled, label1="Close Price",
  title="Results of Normalize Data"
)

In [ ]:
# 3. process inverse normalize features
def inverse(scaler, scaled):
  hasil = scaler.inverse_transform(scaled.reshape(-1,1))
  return hasil

In [ ]:
# 4. process splitting data
train_data, test_data = train_test_split(scaled, train_size=0.80, test_size=0.20, shuffle=False)

In [ ]:
# 4. results splitting data
lineplot_matplotlib2(
  x1=dataset.iloc[0:len(train_data),0], y1=train_data, label1="Training data",
  x2=dataset.iloc[len(train_data):len(dataset),0], y2=test_data, label2="Testing data",
  title="Results of Splitting Data"
)

### 4. Supervised Learning

In [ ]:
# function for supervised learning
def create_dataset(look_back, dataset):
    
  # declare variable X and Y
  dataX = []
  dataY = []
  
  # for loop for create supervised learning
  for i in range(look_back, len(dataset)):
    dataX.append(dataset[i-look_back:i, 0])
    dataY.append(dataset[i, 0])
      
  # return value X and Y
  return np.array(dataX), np.array(dataY)
# ----------------------------------------------------------------------------------------

In [ ]:
# process supervised learning
x_train, y_train = create_dataset(60, train_data)
x_test, y_test = create_dataset(60, test_data)

In [ ]:
# reshape input to be [samples, time steps, features]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
print(x_train.shape)

In [ ]:
# reshape input to be [samples, time steps, features]
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
print(x_test.shape)

### 5. Algorithms SBi-LSTM-RNN

In [ ]:
# config algorithms
algorithms="SBi-LSTM-RNN"

In [ ]:
# process predict with LSTM
lstm_model = get_models(algorithms=algorithms, timestep=x_train.shape[1])

In [ ]:
# process predict with LSTM
lstm_history, lstm_predictions = get_predictions(
  model=lstm_model,
  x_train=x_train, y_train=y_train,
  x_test=x_test, y_test=y_test,
)

In [ ]:
# results training and validation 
lineplot_matplotlib1(
  x1=lstm_history.epoch, y1=lstm_history.history["loss"], label1="loss func on training",
  title="Results training and validation on "+algorithms
)

In [ ]:
# inverse scaler predictions
y_test = inverse(scaler, y_test)
lstm_predictions = inverse(scaler, lstm_predictions)

# results predict with LSTM
lineplot_matplotlib3(
  x1=dataset["Date"].iloc[len(y_train)+120:].values, y1=y_test, label1="actual data",
  x2=dataset["Date"].iloc[len(y_train)+120:].values, y2=lstm_predictions, label2="results predictions",
  title="Results of Predictions BTC-USD with "+algorithms
)

In [ ]:
# calculate eror
lstm_r, lstm_p, lstm_mae, lstm_rmse, lstm_mape = evaluate_models(y_test, lstm_predictions)

# show eror
print("Evaluate Models with : "+str(algorithms))
print("-------------------------------")
print("R       : "+str(lstm_r))
print("P-value : "+str(lstm_p))
print("MAE     : "+str(lstm_mae))
print("RMSE    : "+str(lstm_rmse))
print("MAPE    : "+str(lstm_mape))

### 6. Algorithms SBi-GRU-RNN

In [ ]:
# config algorithms
algorithms="SBi-GRU-RNN"

In [ ]:
# process predict with GRU
gru_model = get_models(algorithms=algorithms, timestep=x_train.shape[1])

In [ ]:
# process predict with GRU
gru_history, gru_predictions = get_predictions(
  model=gru_model,
  x_train=x_train, y_train=y_train,
  x_test=x_test, y_test=y_test,
)

In [ ]:
# results training and validation 
lineplot_matplotlib1(
  x1=gru_history.epoch, y1=gru_history.history["loss"], label1="loss func on training",
  title="Results training and validation on "+algorithms
)

In [ ]:
# inverse scaler predictions
y_test = inverse(scaler, y_test)
gru_predictions = inverse(scaler, gru_predictions)

# results predict with GRU
lineplot_matplotlib3(
  x1=dataset["Date"].iloc[len(y_train)+120:].values, y1=y_test, label1="actual data",
  x2=dataset["Date"].iloc[len(y_train)+120:].values, y2=gru_predictions, label2="results predictions",
  title="Results of Predictions BTC-USD with "+algorithms
)


In [ ]:
# calculate eror
gru_r, gru_p, gru_mae, gru_rmse, gru_mape = evaluate_models(y_test, gru_predictions)

# show eror
print("Evaluate Models with : "+str(algorithms))
print("-------------------------------")
print("R       : "+str(gru_r))
print("P-value : "+str(gru_p))
print("MAE     : "+str(gru_mae))
print("RMSE    : "+str(gru_rmse))
print("MAPE    : "+str(gru_mape))